**Popularity-Destination Collaborative Filtering**

In [3]:
import pandas as pd

# 1. Load popularity scores and test set
popularity = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/features_with_cf_score.csv")
test = pd.read_csv("test_imputed_CSV.csv")

# 2. Keep only necessary columns for merge
popularity_trimmed = popularity[['srch_destination_id', 'prop_id', 'cf_score']]
test_trimmed = test[['srch_id', 'srch_destination_id', 'prop_id']]

# 3. Merge popularity scores into test set
test_scored = test_trimmed.merge(popularity_trimmed, on=['srch_destination_id', 'prop_id'], how='left', sort=False)
# If hotel wasn't seen in training, assign 0 popularity 
# (reasonable fallback in a baseline model — if a hotel was never clicked or booked before, it likely shouldn't be ranked highly.)
test_scored['cf_score'] = test_scored['cf_score'].fillna(0)

# 4. Sort by srch_id and cf_score descending
test_sorted = test_scored.sort_values(by=['srch_id', 'cf_score'], ascending=[True, False])

# 5. Create Kaggle-style submission file
submission = test_sorted[['srch_id', 'prop_id']]
submission.to_csv("assignment_2/results/popularity_cf_submission.csv", index=False)
print("✅ Submission file saved as 'assignment_2/results/popularity_cf_submission.csv'")


MemoryError: Unable to allocate 3.47 GiB for an array with shape (465793408,) and data type int64

In [4]:
import pandas as pd

# Load popularity scores
popularity = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/features_with_cf_score.csv")
popularity_trimmed = popularity[['srch_destination_id', 'prop_id', 'cf_score']]

# Load test set
test = pd.read_csv("test_imputed_CSV.csv")
test_trimmed = test[['srch_id', 'srch_destination_id', 'prop_id']]

# Prepare final result list
chunked_results = []

# Process in chunks of srch_ids
unique_srch_ids = test_trimmed['srch_id'].unique()
batch_size = 10000  # adjust this to fit your memory capacity

print(f"Processing {len(unique_srch_ids)} unique srch_ids in batches...")

for i in range(0, len(unique_srch_ids), batch_size):
    batch_ids = unique_srch_ids[i:i+batch_size]
    test_chunk = test_trimmed[test_trimmed['srch_id'].isin(batch_ids)]

    # Merge only this chunk with popularity scores
    chunk_scored = test_chunk.merge(popularity_trimmed, on=['srch_destination_id', 'prop_id'], how='left', sort=False)
    chunk_scored['cf_score'] = chunk_scored['cf_score'].fillna(0)

    chunked_results.append(chunk_scored)

# Concatenate all results
test_scored = pd.concat(chunked_results, ignore_index=True)

# Sort and save
test_sorted = test_scored.sort_values(by=['srch_id', 'cf_score'], ascending=[True, False])
submission = test_sorted[['srch_id', 'prop_id']]
submission.to_csv("assignment_2/results/popularity_cf_submission.csv", index=False)
print("✅ Submission file saved as 'assignment_2/results/popularity_cf_submission.csv'")


Processing 199549 unique srch_ids in batches...


MemoryError: Unable to allocate 533. MiB for an array with shape (3, 23294822) and data type int64

In [5]:
import pandas as pd

# Load precomputed popularity scores
popularity = pd.read_csv("/Users/nikak/Desktop/vubaby/DMT/data_DMT/features_with_cf_score.csv")
popularity_trimmed = popularity[['srch_destination_id', 'prop_id', 'cf_score']]

# Prepare CSV writer
output_path = "assignment_2/results/popularity_cf_submission.csv"
header_written = False

# Process test set in chunks
chunk_size = 100_000
reader = pd.read_csv("test_imputed_CSV.csv", chunksize=chunk_size)

for i, chunk in enumerate(reader):
    print(f"Processing chunk {i+1}...")
    
    test_chunk = chunk[['srch_id', 'srch_destination_id', 'prop_id']]
    merged = test_chunk.merge(popularity_trimmed, on=['srch_destination_id', 'prop_id'], how='left', sort=False)
    merged['cf_score'] = merged['cf_score'].fillna(0)

    # Sort chunk
    merged_sorted = merged.sort_values(by=['srch_id', 'cf_score'], ascending=[True, False])
    submission_chunk = merged_sorted[['srch_id', 'prop_id']]

    # Append to output file
    submission_chunk.to_csv(output_path, mode='a', header=not header_written, index=False)
    header_written = True

print("✅ Submission file saved at:", output_path)


MemoryError: Unable to allocate 454. MiB for an array with shape (12, 4958347) and data type float64

- --> Hit a Memory Error during the merge, so that is why I had to trim the test set